In [27]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import gmaps
import gmaps.datasets

# Import the requests library
import requests

# Import the API key
from config import weather_api_key
from config import g_key

# Import the datetime module from the datetime library.
from datetime import datetime

import time

In [2]:
# import the csv
weatherpy_df = pd.read_csv("Weather_Data/WeatherPy_vacation.csv")
weatherpy_df.head(10)

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
0,2,Mahebourg,MU,75.20,scattered clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa
1,5,Vaini,TO,78.80,broken clouds,-21.20,-175.20,Keleti Beach Resort
2,9,Kahului,US,82.40,few clouds,20.89,-156.47,Maui Seaside Hotel
3,13,Kapaa,US,80.60,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
4,15,Souillac,MU,75.20,scattered clouds,-20.52,57.52,Bils' Villa
5,16,Warri,NG,81.21,overcast clouds,5.52,5.75,BON Hotel Delta
6,17,Mabaruma,GY,75.78,scattered clouds,8.20,-59.78,NaN
7,21,Rikitea,PF,76.37,few clouds,-23.12,-134.97,Pension Maro'i
8,22,Jibuti,DJ,84.20,clear sky,11.50,42.50,Campement touristique DANKALELO
9,32,Kidal,ML,84.63,scattered clouds,18.44,1.41,NaN


In [66]:
# Recreating the marker map (without heat layer) to choose the 4 cities in close proximity

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt> Current Weather</dt><dd>{Weather} and {Max Temp}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in weatherpy_df.iterrows()]

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Add a heatmap of temperature for the vacation spots
locations = weatherpy_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# create the DataFrames for the 4 chosen cities - Georgetown, Ranong,Quang Ngai,Si Bun Ruang   
georgetown_df = weatherpy_df.loc[weatherpy_df["City"]=="Georgetown"]
ranong_df = weatherpy_df.loc[weatherpy_df["City"]=="Ranong"]
sibunruang_df = weatherpy_df.loc[weatherpy_df["City"]=="Si Bun Ruang"]
quangngai_df = weatherpy_df.loc[weatherpy_df["City"]=="Quang Ngai"] 
georgetown_df

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
38,173,Georgetown,MY,82.4,few clouds,5.41,100.34,Cititel Penang


In [5]:
ranong_df

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
63,267,Ranong,TH,80.24,overcast clouds,9.97,98.63,Tinidee Hotel@Ranong


In [6]:
sibunruang_df

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
22,86,Si Bun Ruang,TH,81.16,overcast clouds,16.97,102.28,รีสอร์ทบ้านสวนอินน์ BANSUAN HOTEL


In [7]:
quangngai_df

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
109,473,Quang Ngai,VN,76.3,overcast clouds,15.12,108.8,Central Hotel


In [49]:
# get each city's lat-long pairs
georgetown_np = georgetown_df.to_numpy()
georgetown = [georgetown_np.item(5),georgetown_np.item(6)]

ranong_np = ranong_df.to_numpy()
ranong = [ranong_np.item(5), ranong_np.item(6)]

sibunruang_np = sibunruang_df.to_numpy()
sibunruang = [sibunruang_np.item(5), sibunruang_np.item(6)]

quangngai_np = quangngai_df.to_numpy()
quangngai = [quangngai_np.item(5), quangngai_np.item(6)]

In [59]:
# Get the travel routes map linking the 4 cities

gmaps.configure(api_key=g_key)


fig = gmaps.figure()
gt_rg_sb_qg = gmaps.directions_layer(georgetown,ranong, waypoints=[sibunruang,quangngai], travel_mode="DRIVING" )

fig.add_layer(gt_rg_sb_qg)


fig

Figure(layout=FigureLayout(height='420px'))

In [63]:
# Create a new DataFrame with the 4 cities
four_cities = [georgetown_df,ranong_df,sibunruang_df,quangngai_df]
four_cities_df = pd.concat(four_cities)
four_cities_df

,City_ID,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
38,173,Georgetown,MY,82.40,few clouds,5.41,100.34,Cititel Penang
63,267,Ranong,TH,80.24,overcast clouds,9.97,98.63,Tinidee Hotel@Ranong
22,86,Si Bun Ruang,TH,81.16,overcast clouds,16.97,102.28,รีสอร์ทบ้านสวนอินน์ BANSUAN HOTEL
109,473,Quang Ngai,VN,76.30,overcast clouds,15.12,108.80,Central Hotel


In [75]:
# Create the marker maps wioth the 4 cities

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt> Current Weather</dt><dd>{Weather} and {Max Temp}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in four_cities_df.iterrows()]

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Add a heatmap of temperature for the vacation spots
locations = four_cities_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(12, 105), zoom_level=5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))